In [1]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
#from sklearn.linear_model import LinearRegression
#import statsmodels.api as sm
#from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV,cross_val_score,cross_val_predict
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('HR.csv')
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low


In [13]:
df.shape

(14999, 10)

In [14]:
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11991 entries, 0 to 11990
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     11991 non-null  float64
 1   last_evaluation        11991 non-null  float64
 2   number_project         11991 non-null  int64  
 3   average_montly_hours   11991 non-null  int64  
 4   time_spend_company     11991 non-null  int64  
 5   Work_accident          11991 non-null  int64  
 6   left                   11991 non-null  int64  
 7   promotion_last_5years  11991 non-null  int64  
 8   department             11991 non-null  object 
 9   salary                 11991 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 936.9+ KB


In [16]:
pd.isnull(df).sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
department               0
salary                   0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder # For change categorical variable into int

from sklearn.metrics import accuracy_score 

le=LabelEncoder()

df['salary']=le.fit_transform(df['salary'])

In [18]:
def dummies(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df
df = dummies('department',df)

In [19]:
from sklearn.neighbors import LocalOutlierFactor
def outlier_detection(df):
    x = df.drop("left", axis=1)
    y = df.left
    lof =LocalOutlierFactor(n_neighbors = 10, metric= "euclidean") #or manhattan

    res = lof.fit_predict(x)
    x_o = x[res != -1]
    y_o = y[res != -1]
    df = pd.concat([x_o, y_o], axis=1)
    return df
df_cleaned= outlier_detection(df)

In [20]:
df['salary'].unique()

array([1, 2, 0])

In [21]:
X=df

In [22]:
def knn_imputing(X):
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=5)
    imputer.fit_transform(X)
    return X
df_filled = knn_imputing(df_cleaned)

In [23]:
df_filled

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,left
0,0.38,0.53,2,157,3,0,0,1,0,0,0,0,0,0,1,0,0,1
1,0.80,0.86,5,262,6,0,0,2,0,0,0,0,0,0,1,0,0,1
2,0.11,0.88,7,272,4,0,0,2,0,0,0,0,0,0,1,0,0,1
3,0.72,0.87,5,223,5,0,0,1,0,0,0,0,0,0,1,0,0,1
4,0.37,0.52,2,159,3,0,0,1,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11986,0.90,0.55,3,259,10,1,1,0,0,0,0,1,0,0,0,0,0,0
11987,0.74,0.95,5,266,10,0,1,0,0,0,0,1,0,0,0,0,0,0
11988,0.85,0.54,3,185,10,0,1,0,0,0,0,1,0,0,0,0,0,0
11989,0.33,0.65,3,172,10,0,1,0,0,0,0,0,1,0,0,0,0,0


In [22]:
def change_name_columns(df_old,df_new):
    name_columns=list(df_old.columns)
    df_new.columns=name_columns
    return df_new
df_filled=change_name_columns(df,df_filled)

In [24]:
num = [col for col in df.columns if df[col].dtype!='O']

In [25]:
df[num]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.38,0.53,2,157,3,0,1,0,1,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,2,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,2,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,0,1,0,1,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,0,1,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11986,0.90,0.55,3,259,10,1,0,1,0,0,0,0,1,0,0,0,0,0
11987,0.74,0.95,5,266,10,0,0,1,0,0,0,0,1,0,0,0,0,0
11988,0.85,0.54,3,185,10,0,0,1,0,0,0,0,1,0,0,0,0,0
11989,0.33,0.65,3,172,10,0,0,1,0,0,0,0,0,1,0,0,0,0


In [28]:
need = ['number_project','average_montly_hours','time_spend_company']
scaler = MinMaxScaler()
df_num=scaler.fit_transform(df_cleaned[need])
df_num=pd.DataFrame(data=df_num)

In [29]:
def change_name_columns(df_old):
    need = ['number_project','average_montly_hours','time_spend_company']
    df_old.columns=need
    return df_old
df_num=change_name_columns(df_num)
df_num.head()

,number_project,average_montly_hours,time_spend_company
0,0.0,0.285047,0.125
1,0.6,0.775701,0.500
2,1.0,0.822430,0.250
3,0.6,0.593458,0.375
4,0.0,0.294393,0.125


In [30]:
df_cleaned[need]=df_num

In [31]:
df_cleaned.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,left
0,0.38,0.53,0.0,0.285047,0.125,0,0,1,0,0,0,0,0,0,1,0,0,1
1,0.80,0.86,0.6,0.775701,0.500,0,0,2,0,0,0,0,0,0,1,0,0,1
2,0.11,0.88,1.0,0.822430,0.250,0,0,2,0,0,0,0,0,0,1,0,0,1
3,0.72,0.87,0.6,0.593458,0.375,0,0,1,0,0,0,0,0,0,1,0,0,1
4,0.37,0.52,0.0,0.294393,0.125,0,0,1,0,0,0,0,0,0,1,0,0,1


In [32]:
df_cleaned.to_csv('HR_Preprocessing.csv')